# Import necessary packages

In [123]:
import copy
import sympy as sp
from sympy.matrices import ones, eye
import re
import dimod
import neal
from dwave.system import DWaveSampler, EmbeddingComposite

# Input Variables

coords_original - Coordinates considered for distance measure in the hubo<br>torsional_bonds - Torsional bond numbers and their respective coordinate ends<br>coords_rotation_dict - Coordinate numbers and the bonds that affect them<br>distance_pairs - Pairs of coordinates to find distance

In [124]:
coords_original = {
    0: [1.0, -0.5, 0.0],
    1: [0.0, 0.0, 0.0],
    2: [0.0, 1.0, 0.0],
    3: [1.0, 1.5, 0.0],
    4: [2.0, 1.0, 0.0]
}
torsional_bonds = {
    0: (1, 2),
    1: (2, 3)
}
coords_rotation_dict = {
    0: [],
    1: [],
    2: [],
    3: [0],
    4: [0, 1]
}
distance_pairs = [
    (0, 3),
    (0, 4),
    (1, 3),
    (1, 4),
    (2, 4)]

In [125]:
coords_dict = copy.deepcopy(coords_original)
final_coords = copy.deepcopy(coords_original)

n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 4  # no. of discrete angles

x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7)


# Function to generate hard constraint

In [126]:
def generate_hard_constraint(include_a=True, a_value=1.0):
    for i in range(n_bonds):
        x.append(sp.symbols(f'x_{str(i)}_(0:{n_angles})'))
    hard_constraint = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[i][j]
        hard_constraint += (summation - 1) ** 2
    if include_a:
        a_const = sp.Symbol('A_const')
        hard_constraint *= a_const
    else:
        hard_constraint *= a_value
    print(x)
    return hard_constraint.expand()

In [127]:
def generate_hard_constraint_2():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

In [128]:
hard_hubo = generate_hard_constraint_2()
sp.pprint(hard_hubo)

        ⎛                       2                          2⎞
A_const⋅⎝(x₀ + x₁ + x₂ + x₃ - 1)  + (x₄ + x₅ + x₆ + x₇ - 1) ⎠


# Functions to rotate coordinates with hubo variables

In [129]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [130]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [131]:
def generate_rotation_matrix_2(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [132]:
def generate_rotation_matrix(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = sp.expand(dx ** 2 + dy ** 2 + dz ** 2)
    l = sp.expand(sp.sqrt(l_sq))
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    for i in range(n_angles):
        c_theta += sp.expand(sp.cos(thetas[i]) * x[bond_no][i])
        s_theta += sp.expand(sp.sin(thetas[i]) * x[bond_no][i])
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = sp.expand((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq)
    rotation_matrix[0, 1] = sp.expand((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq)
    rotation_matrix[0, 2] = sp.expand((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq)
    rotation_matrix[0, 3] = sp.expand(((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq)
    rotation_matrix[1, 0] = sp.expand((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq)
    rotation_matrix[1, 1] = sp.expand((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq)
    rotation_matrix[1, 2] = sp.expand((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq)
    rotation_matrix[1, 3] = sp.expand(((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq)
    rotation_matrix[2, 0] = sp.expand((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq)
    rotation_matrix[2, 1] = sp.expand((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq)
    rotation_matrix[2, 2] = sp.expand((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq)
    rotation_matrix[2, 3] = sp.expand(((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq)
    return sp.expand(rotation_matrix)

In [133]:
def rotate_all_coordinates():
    for i in coords_original.keys():
        if len(coords_rotation_dict[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict[i]:
                temp_rot_mat = generate_rotation_matrix_2(coords_original[torsional_bonds[bond_no][0]],
                                                        coords_original[torsional_bonds[bond_no][1]], bond_no)
                rot_mat = rot_mat * temp_rot_mat
            coords_dict[i] = rotate_coordinates(rot_mat, coords_original[i])

In [134]:
rotate_all_coordinates()

In [135]:
sp.pprint(coords_original)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0, 1.5, 0.
0], 4: [2.0, 1.0, 0.0]}


In [136]:
sp.pprint(coords_dict)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0⋅x₀ - x₂,
 1.5, -x₁ + 1.0⋅x₃], 4: [0.4⋅x₀⋅x₄ - 0.4⋅x₀⋅x₆ + 1.6⋅x₀ - 0.894427190999916⋅x₁
⋅x₅ + 0.894427190999916⋅x₁⋅x₇ - 0.4⋅x₂⋅x₄ + 0.4⋅x₂⋅x₆ - 1.6⋅x₂ + 0.89442719099
9916⋅x₃⋅x₅ - 0.894427190999916⋅x₃⋅x₇, -0.8⋅x₄ + 0.8⋅x₆ + 1.8, -0.8944271909999
16⋅x₀⋅x₅ + 0.894427190999916⋅x₀⋅x₇ - 0.4⋅x₁⋅x₄ + 0.4⋅x₁⋅x₆ - 1.6⋅x₁ + 0.894427
190999916⋅x₂⋅x₅ - 0.894427190999916⋅x₂⋅x₇ + 0.4⋅x₃⋅x₄ - 0.4⋅x₃⋅x₆ + 1.6⋅x₃]}


# Functions to generate the optimization contraint in hubo

In [137]:
def distance_squared(first_coords, second_coords):
    dis_sq = sp.expand((second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 +(second_coords[2] - first_coords[2])**2)
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_dict[pair[0]], coords_dict[pair[1]])
    return distance_sq.expand()

In [138]:
distance_hubo = generate_distance_hubo()
sp.pprint(distance_hubo)

       2   2          2                2            2   2         2           
0.48⋅x₀ ⋅x₄  - 0.96⋅x₀ ⋅x₄⋅x₆ + 3.84⋅x₀ ⋅x₄ + 2.4⋅x₀ ⋅x₅  - 4.8⋅x₀ ⋅x₅⋅x₇ + 0.

     2   2          2            2   2          2                2            
48⋅x₀ ⋅x₆  - 3.84⋅x₀ ⋅x₆ + 2.4⋅x₀ ⋅x₇  + 9.68⋅x₀  - 0.96⋅x₀⋅x₂⋅x₄  + 1.92⋅x₀⋅x

                                      2                                  2    
₂⋅x₄⋅x₆ - 7.68⋅x₀⋅x₂⋅x₄ - 4.8⋅x₀⋅x₂⋅x₅  + 9.6⋅x₀⋅x₂⋅x₅⋅x₇ - 0.96⋅x₀⋅x₂⋅x₆  + 7

                           2                                                  
.68⋅x₀⋅x₂⋅x₆ - 4.8⋅x₀⋅x₂⋅x₇  - 19.36⋅x₀⋅x₂ - 0.8⋅x₀⋅x₄ + 0.8⋅x₀⋅x₆ - 5.2⋅x₀ + 

       2   2          2                2            2   2         2           
0.48⋅x₁ ⋅x₄  - 0.96⋅x₁ ⋅x₄⋅x₆ + 3.84⋅x₁ ⋅x₄ + 2.4⋅x₁ ⋅x₅  - 4.8⋅x₁ ⋅x₅⋅x₇ + 0.

     2   2          2            2   2          2                2            
48⋅x₁ ⋅x₆  - 3.84⋅x₁ ⋅x₆ + 2.4⋅x₁ ⋅x₇  + 9.68⋅x₁  - 0.96⋅x₁⋅x₃⋅x₄  + 1.92⋅x₁⋅x

                                      2       

# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [139]:
hubo_expr = sp.expand(hard_hubo - distance_hubo)
hubo_expr

A_const*x0**2 + 2*A_const*x0*x1 + 2*A_const*x0*x2 + 2*A_const*x0*x3 - 2*A_const*x0 + A_const*x1**2 + 2*A_const*x1*x2 + 2*A_const*x1*x3 - 2*A_const*x1 + A_const*x2**2 + 2*A_const*x2*x3 - 2*A_const*x2 + A_const*x3**2 - 2*A_const*x3 + A_const*x4**2 + 2*A_const*x4*x5 + 2*A_const*x4*x6 + 2*A_const*x4*x7 - 2*A_const*x4 + A_const*x5**2 + 2*A_const*x5*x6 + 2*A_const*x5*x7 - 2*A_const*x5 + A_const*x6**2 + 2*A_const*x6*x7 - 2*A_const*x6 + A_const*x7**2 - 2*A_const*x7 + 2*A_const - 0.48*x0**2*x4**2 + 0.96*x0**2*x4*x6 - 3.84*x0**2*x4 - 2.4*x0**2*x5**2 + 4.8*x0**2*x5*x7 - 0.48*x0**2*x6**2 + 3.84*x0**2*x6 - 2.4*x0**2*x7**2 - 9.68*x0**2 + 0.96*x0*x2*x4**2 - 1.92*x0*x2*x4*x6 + 7.68*x0*x2*x4 + 4.8*x0*x2*x5**2 - 9.6*x0*x2*x5*x7 + 0.96*x0*x2*x6**2 - 7.68*x0*x2*x6 + 4.8*x0*x2*x7**2 + 19.36*x0*x2 + 0.8*x0*x4 - 0.8*x0*x6 + 5.2*x0 - 0.48*x1**2*x4**2 + 0.96*x1**2*x4*x6 - 3.84*x1**2*x4 - 2.4*x1**2*x5**2 + 4.8*x1**2*x5*x7 - 0.48*x1**2*x6**2 + 3.84*x1**2*x6 - 2.4*x1**2*x7**2 - 9.68*x1**2 + 0.96*x1*x3*x4**2 - 1.9

In [157]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        monom_key = monom.as_coeff_mul()[1][1:]
        dict_key = []
        for monom_item in monom_key:
            if re.match("^x(\d*)\d$", str(monom_item)):
                dict_key.append(int(str(monom_item)[1:]))
            else:
                dict_key.append(int(str(monom_item).split('**')[0][1:]))
        if len(dict_key) > 0:
            dict_key.sort()
            dict_key = tuple(dict_key)
        else:
            dict_key = ()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += monom.as_coeff_mul()[1][0]*monom.as_coeff_mul()[0]
        else:
            hubo_dict[dict_key] = monom.as_coeff_mul()[1][0]*monom.as_coeff_mul()[0]
    return hubo_dict

In [158]:
hubo_dict = hubo_expr_to_dict()
print(hubo_dict)

{(): 2*A_const - 17.42, (0,): -A_const - 4.48, (4,): 5.92 - A_const, (2,): -A_const - 14.88, (6,): -A_const - 9.76, (1,): -A_const - 9.68, (3,): -A_const - 9.68, (5,): -A_const, (7,): -A_const, (4, 6): 2*A_const + 3.84, (0, 6): 2.56000000000000, (1, 6): 3.36000000000000, (2, 6): 4.16000000000000, (3, 6): 3.36000000000000, (0, 2): 2*A_const + 19.36, (1, 3): 2*A_const + 19.36, (0, 4): -3.52000000000000, (1, 7): -0.611145618000169, (3, 5): -0.611145618000169, (1, 4): -4.32000000000000, (2, 4): -5.12000000000000, (3, 4): -4.32000000000000, (0, 5): -2.40000000000000, (0, 7): -2.40000000000000, (1, 5): -4.18885438199983, (2, 5): -2.40000000000000, (2, 7): -2.40000000000000, (3, 7): -4.18885438199983, (0, 1): 2*A_const, (0, 3): 2*A_const, (1, 2): 2*A_const, (2, 3): 2*A_const, (4, 5): 2*A_const, (4, 7): 2*A_const, (5, 6): 2*A_const, (5, 7): 2*A_const, (6, 7): 2*A_const, (0, 2, 4): 8.64000000000000, (0, 2, 6): -6.72000000000000, (1, 3, 4): 8.64000000000000, (1, 3, 6): -6.72000000000000, (0, 4, 

In [159]:
file_name = "hubo_dict.txt"
f = open(file_name, "w")
print('full hubo dictionary written in file - hubo_dict.txt')
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - hubo_dict.txt


In [160]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(file_name, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(file_name, 'r').read()
HUBO=eval(read_dictionary)

In [161]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 57


In [162]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [163]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 21


In [164]:
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [165]:
sampler = neal.SimulatedAnnealingSampler()

sample_size=10

In [166]:
sampleset = sampler.sample(bqm, num_reads=sample_size)
print(sampleset)

   0  1  2  3  4  5  6  7 energy num_oc.
0  0  0  1  0  0  0  0  1  -32.3       1
5  0  0  1  0  0  0  0  1  -32.3       1
7  0  0  1  0  0  0  0  1  -32.3       1
9  0  0  1  0  0  1  0  0  -32.3       1
3  1  0  0  0  0  0  1  0 -31.66       1
2  0  0  0  1  0  1  0  0  -27.1       1
6  0  0  0  1  0  1  0  0  -27.1       1
8  0  0  0  1  0  1  0  0  -27.1       1
1  1  0  0  0  0  0  0  1  -21.9       1
4  0  0  0  1  1  0  0  0 -21.18       1
['BINARY', 10 rows, 10 samples, 8 variables]


In [167]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=1000)
print(sampleset.first.sample)

{0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0}
